In [1]:
s = "def greeting(name: str) -> str: \
    return 'Hello ' + name "

In [2]:
import astor,ast

In [3]:
a = ast.parse(s)

In [4]:
#!pipenv install typed-ast

In [5]:
from typed_ast import ast3

In [6]:
a = ast3.parse(s)

In [7]:
a

In [8]:
hypothesis_mappings = {"str": {"decorator":"@given(text())","hypothesis_test":"s"}}

def make_test(a):
    for i in a.body:
        name = i.name
        r = i.returns.id
        args = [ x.arg for x in i.args.args]
        args = ", ".join(args)
        hypothesis_test = hypothesis_mappings[r]["hypothesis_test"]
        t = hypothesis_mappings[r]["decorator"]+"\n"
        t += f'def test_{name}({hypothesis_test}):\n'
        t +=f'    assert type({name}({hypothesis_test})) is {r}'
        return t

In [9]:
print(make_test(a))

@given(text())
def test_greeting(s):
    assert type(greeting(s)) is str


In [10]:
#!pipenv install hypothesis

In [11]:
from hypothesis import given
from hypothesis.strategies import text

def greeting(name: str) -> str: 
    return 'Hello ' + name 
@given(text())
def test_greeting(s):
    assert type(greeting(s)) is str

In [12]:
test_greeting()

In [13]:
data={"datasets": [{"label": "data1", "data": [{"x": 1, "y": 1}, {"x": 2, "y": 2}]}]}

In [14]:
s=open("t_t.py").read()
a = ast3.parse(s)

In [15]:
a.body

In [16]:
import json
from collections import namedtuple

In [44]:
from typing import List
class DataPoint:
    def __init__(self,x:int,y:int):
        self.x = x
        self.y =y

class DataSet:
    def __init__(self,label:str,data:List[DataPoint],backgroundColor:str):
        self.label=label
        self.data=data
        self.backgroundColor = backgroundColor

class Data:
    def __init__(self,datasets:List[DataSet]=None):
        self.datasets=datasets

In [46]:
data= {"datasets": [{"label": "data1", "data": [{"x": 1, "y": 1}, {"x": 2, "y": 2}]}]}
data1 = Data([DataSet(label="label1",data=[DataPoint(1,2),DataPoint(3,4)],backgroundColor="00b8ff")])

In [75]:
zzz = list(data1.__dict__.values())
for i in zzz:
    if type(i) is type or list:
        print(type(i))
    else:
        print(f'Error: {i}')

<class 'list'>


In [97]:
recursive_json(data1)

['label1']

In [98]:
[ i.__dict__ for i in data1.datasets[0].data ] 

[{'x': 1, 'y': 2}, {'x': 3, 'y': 4}]

In [81]:
data1.__dict__

{'datasets': [<__main__.DataSet at 0x7f1ccff136a0>]}

In [ ]:
for dataset in data:                                                                           
    #dataset = [i.as_dict() for i in dataset]
    dataset

In [100]:
#!pipenv install jsons

In [106]:
import jsons
json.loads(jsons.dumps(data1))

{'datasets': [{'backgroundColor': '00b8ff',
   'data': [{'x': 1, 'y': 2}, {'x': 3, 'y': 4}],
   'label': 'label1'}]}